# Sentiment Analysis and Stock Prediction Model

## Retrieve data

In [1]:
import stock_retriever as sr
from datetime import datetime
import os
stocks = ['MSFT', 'AAPL', 'NVDA','GOOGL','GOOG','AMZN','META','TSLA']
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')
for stock in stocks:
    if(not os.path.exists(f'stock_data/{stock}_{start_date}_{end_date}.csv')):
        sr.get_stock_data(stock, start_date=start_date, end_date=end_date).to_csv(f'stock_data/{stock}_{start_date}_{end_date}.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## Deep Learning

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Generate synthetic stock price data (replace with real data)
np.random.seed(42)
n_days = 100
stock_prices = np.cumsum(np.random.randn(n_days))

# Normalize data
normalized_prices = (stock_prices - np.min(stock_prices)) / (np.max(stock_prices) - np.min(stock_prices))

# Prepare input sequences (sliding window approach)
seq_length = 10
X, y = [], []
for i in range(len(normalized_prices) - seq_length):
    X.append(normalized_prices[i:i + seq_length])
    y.append(normalized_prices[i + seq_length])

X = torch.tensor(X).unsqueeze(2).float()
y = torch.tensor(y).float()

# Define LSTM model
class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = 1
hidden_size = 32
output_size = 1
model = StockLSTM(input_size, hidden_size, output_size)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch + 1}/{n_epochs}], Loss: {loss.item():.4f}")

# Make predictions
model.eval()
with torch.inference_mode():
    future_days = 10
    future_inputs = X[-1, -seq_length:].unsqueeze(0)
    for _ in range(future_days):
        future_output = model(future_inputs)
        future_inputs = torch.cat((future_inputs[:, 1:], future_output.unsqueeze(1)), dim=1)

    predicted_prices = future_inputs.squeeze().numpy()

# Plot results
plt.plot(stock_prices, label="Actual Prices")
plt.plot(np.arange(n_days, n_days + future_days), predicted_prices, label="Predicted Prices", linestyle="--")
plt.xlabel("Days")
plt.ylabel("Normalized Prices")
plt.title("Stock Price Prediction with LSTM")
plt.legend()
plt.show()
